# Command Pattern

## Intent

> Command is a behavioral design pattern that turns a request into a stand-alone object that contains all information about the request. This transformation lets you pass requests as a method arguments, delay or queue a request’s execution, and support undoable operations. (refactoring)

重點

* 把請求封裝成物件
* 延遲執行
* undoable

## Applicability

同上一段的重點。

## Structure

![img](https://refactoring.guru/images/patterns/diagrams/command/structure.png?id=1cd7833638f4c43630f4a84017d31195)

(圖片來源: refactoring.guru)

## Example

### before using command pattern

拿cb交辦任務給我作為例子，如果沒有套用command pattern的情況下。

```mermaid
sequenceDiagram
    Actor  cb
    Actor  rockefel
    cb ->> rockefel : 做A功能
    rockefel ->> rockefel : 完成A功能
    cb ->> rockefel : 做B功能
    rockefel ->> rockefel : 完成B功能
```

![image](https://i.imgur.com/0hdHi1J.png)

In [ ]:
public class Rockefel{
    public void DoA() => Console.WriteLine("rockefel: 完成A功能");
    public void DoB() => Console.WriteLine("rockefel: 完成B功能");
}

public class CB{
    public void Command(){
        Rockefel rockefel = new();
        Console.WriteLine("cb: 去做A功能");
        rockefel.DoA();
        Console.WriteLine("cb: 去做B功能");
        rockefel.DoB();
    }
}

new CB().Command();

cb: 去做A功能
rockefel: 完成A功能
cb: 去做B功能
rockefel: 完成B功能


### using command pattern

接著套上 command pattern 來修改上面的例子。

首先，我不希望command會像是"做A功能"、"做B功能"這樣子，為了這些只能用一次的東西新增command class實在有點麻煩。
所以先把command設計成reusable，例如:

* 新增功能(功能)
* 修改功能(功能)
* 刪除功能(功能)
* 重構功能(功能)

重新畫一次sequence diagram

```mermaid
sequenceDiagram
    Actor  cb as cb(Client)
    Participant iv as Invoker
    Actor  rockefel as rockefel(Receiver)
    cb ->> cb : 建立新的Command"新增功能A"並設定Receiver為rockefel
    cb ->> cb : 建立新的Command"新增功能B"並設定Receiver為rockefel
    cb ->> iv : 把先前新增的Command加入queue。
    cb ->> iv : 執行命令
    iv ->> rockefel : 依序傳入Command並執行
```

![sd](https://i.imgur.com/FUj8BS0.png)

先把被Invoker以及Client依賴的Receiver建立出來

In [ ]:
// receiver
public class Rockefel{
    public void AddFeature(string feature) => Console.WriteLine($"rockefel 新增了功能:{feature}");
    public void ModifyFeature(string feature) => Console.WriteLine($"rockefel 修改了功能:{feature}");
    public void DeleteFeature(string feature) => Console.WriteLine($"rockefel 刪除了功能:{feature}");
    public void RefactorFeature(string feature) => Console.WriteLine($"rockefel 重構了功能:{feature}");
}

接著來實作 `Command`

這邊就要根據undo的做法而有所分別。

這邊提供一些undo實作的思路供參考:

1. 把undo寫成命令。`XXXCommand` -> `UndoXXXCommand`
2. 在interface中強制每個命令實作`undo`方法。

這個範例中我採用第二種作法。

In [ ]:
public interface ICommand{
    public void Do();
    public void Undo();
}

public class AddFeature:ICommand{
    Rockefel rockefel;
    public AddFeature(Rockefel rockefel){
        this.rockefel = rockefel;
    }
    public void Do(){}
    public void Undo(){}
}


復原的方式一樣提供兩種思路:

1. 在Receiver寫好復原的邏輯。
2. 套用 Memento Pattern。

這邊使用第一種做法。
稍微修正一下先前的設計。

In [ ]:
// receiver
public class Rockefel{
    public void AddFeature(string feature) => Console.WriteLine($"rockefel 新增了功能:{feature}");
    public void UndoAddFeature(string feature) => Console.WriteLine($"rockefel 還原了先前新增的功能:{feature}");
    public void ModifyFeature(string feature) => Console.WriteLine($"rockefel 修改了功能:{feature}");
    public void UndoModifyFeature(string feature) => Console.WriteLine($"rockefel 還原了先前修改的功能:{feature}");
    public void DeleteFeature(string feature) => Console.WriteLine($"rockefel 刪除了功能:{feature}");
    public void UndoDeleteFeature(string feature) => Console.WriteLine($"rockefel 還原了先前刪除的功能:{feature}");
    public void RefactorFeature(string feature) => Console.WriteLine($"rockefel 重構了功能:{feature}");
    public void UndoRefactorFeature(string feature) => Console.WriteLine($"rockefel 還原了先前重構的功能:{feature}");
}

public class AddFeature:ICommand{
    Rockefel rockefel;
    string feature;
    public AddFeature(Rockefel rockefel,string feature){
        this.rockefel = rockefel;
        this.feature = feature;
    }
    public void Do()=> rockefel.AddFeature(feature);
    public void Undo()=> rockefel.UndoAddFeature(feature);
}

public class ModifyFeature:ICommand{
    Rockefel rockefel;
    string feature;
    public ModifyFeature(Rockefel rockefel,string feature){
        this.rockefel = rockefel;
        this.feature = feature;
    }
    public void Do()=> rockefel.ModifyFeature(feature);
    public void Undo()=> rockefel.UndoModifyFeature(feature);
}

public class DeleteFeature:ICommand{
    Rockefel rockefel;
    string feature;
    public DeleteFeature(Rockefel rockefel,string feature){
        this.rockefel = rockefel;
        this.feature = feature;
    }
    public void Do()=> rockefel.DeleteFeature(feature);
    public void Undo()=> rockefel.UndoDeleteFeature(feature);
}

public class RefactorFeature:ICommand{
    Rockefel rockefel;
    string feature;
    public RefactorFeature(Rockefel rockefel,string feature){
        this.rockefel = rockefel;
        this.feature = feature;
    }
    public void Do()=> rockefel.RefactorFeature(feature);
    public void Undo()=> rockefel.UndoRefactorFeature(feature);
}

接著實作`Invoker`

In [ ]:
public class Invoker{
    List<ICommand> commands = new();
    public void SetCommand(ICommand command) => commands.Add(command);
    public void ExecuteCommands() => commands.ForEach(c=>c.Do());
    public void Undo(){
        commands.Reverse();
        commands.ForEach(c=>c.Undo());
    }
}

執行

In [ ]:
Invoker inv = new();
Rockefel rockefel = new();
Console.WriteLine("cb 下達命令");
Console.WriteLine("新增庫存查詢功能");
inv.SetCommand(new AddFeature(rockefel,"庫存查詢"));
Console.WriteLine("修改材料掛載功能");
inv.SetCommand(new ModifyFeature(rockefel,"材料掛載"));
Console.WriteLine("刪除作業員登入功能");
inv.SetCommand(new DeleteFeature(rockefel,"作業員登入"));

Console.WriteLine("cb: 現在開始進行上述任務");
inv.ExecuteCommands();

Console.WriteLine("cb: 需求有變，取消先前的命令並還原程式功能");
inv.Undo();

cb 下達命令
新增庫存查詢功能
修改材料掛載功能
刪除作業員登入功能
cb: 現在開始進行上述任務
rockefel 新增了功能:庫存查詢
rockefel 修改了功能:材料掛載
rockefel 刪除了功能:作業員登入
cb: 需求有變，取消先前的命令並還原程式功能
rockefel 還原了先前刪除的功能:作業員登入
rockefel 還原了先前修改的功能:材料掛載
rockefel 還原了先前新增的功能:庫存查詢


### Class Diagram

![cd](https://i.imgur.com/NNIaklK.png)

## Pros and Cons

Pros:

* 上面提過的那些特點
* 新增Command很容易

Cons:

* 多出很多物件

## References

* [refactoring-command](https://refactoring.guru/design-patterns/command)